In [225]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  
import copy

import tools.Sample_Tools as smpl
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)


import matplotlib.pyplot as plt
import seaborn as sns

# from base.JuUnits import parallal_task,task_chunk_split
# from base.JuUnits import excute_for_multidates

# from sklearn.decomposition import FactorAnalysis
# from factor_analyzer import FactorAnalyzer
# from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity,calculate_kmo

# import inspect
import warnings
from sklearn.model_selection import train_test_split

from catboost import CatBoost
from catboost import CatBoostRanker, Pool, MetricVisualizer


In [16]:
file_names = ['base10','alpha42','alpha54','alpha60','alpha77','191_alpha004', '191_alpha010', '191_alpha021', '191_alpha070', '191_alpha095', '191_alpha115', 
 '191_alpha124', '191_alpha126', '191_alpha129', '191_alpha132', '191_alpha143', '191_alpha144', 
  '191_alpha146', '191_alpha150', '191_alpha153', '191_alpha160', '191_alpha161', '191_alpha173', '191_alpha187']

temp = []
for file in file_names[0:7]:
    print(file)
    temp.append(load_cache(file+'_train',cache_type=CACHE_TYPE.FACTOR)
)

data_all = pd.concat(temp,axis=1).sort_index()#.loc[(slice('2017-01-11','2020-12-24'),codes),:]



In [181]:
data_slice = data_all.loc[(slice('2018-01-11','2020-12-24'),slice(None))]

In [182]:
codes = data_slice.index.get_level_values(1).unique()
dates = data_slice.index.get_level_values(0).unique()
data_replenish = load_cache_adv('all_train_qfq',
                                dates.min().strftime('%Y-%m-%d'),
                                dates.max().strftime('%Y-%m-%d'),
                                cache_type=CACHE_TYPE.STOCK)

data_replenish = data_replenish.loc[(slice(None),data_replenish.index.get_level_values(1).unique().intersection(codes)),]

ret = smpl.get_forward_return(data_replenish,'close')
ret.name = 'target'
ret[np.abs(ret)>0.15]=np.nan

industy = data_replenish['industry']
industy.name = 'industy'

cap_log = np.log(data_replenish['market_value'])
cap_log.name = 'cap_log'


data = pd.concat([data_slice,industy,cap_log,ret],axis=1).sort_index().dropna()



In [183]:
data['industy'] = data['industy'].astype('string')
data['191_alpha004'] = data['191_alpha004'].astype('int')
train_data = data[data.columns[:-1]]
train_target = data[[data.columns[-1]]]


X_train,X_test, y_train, y_test =train_test_split(train_data,train_target,test_size=.2, shuffle=False)
# X_train_group = y_train.groupby(level=0).count()
# X_train.nunique()
X_train_group = y_train.index.get_level_values(0).strftime('%Y-%m-%d').tolist()
y_test_group = y_test.index.get_level_values(0).strftime('%Y-%m-%d').tolist()

In [185]:
# X_train

In [184]:
categorical_features_names = ['191_alpha004', 'industy']
train_p = Pool(
    data=X_train,
    label=y_train,
    group_id=X_train_group,
    cat_features=categorical_features_names
)

test_p = Pool(
    data=X_test,
    label=y_test,
    group_id=y_test_group,
    cat_features=categorical_features_names
)

In [290]:
default_parameters = {
    'learning_rate': .05,
    'iterations': 50,
    'custom_metric': None,
    'verbose': False,
    'random_seed': 0,
    'use_best_model':True,
    'early_stopping_rounds': 25,
}



def fit_model(loss_function, train_pool, test_pool, additional_params=None, verbose=False):
    parameters = copy.deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    parameters['verbose'] = verbose
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    # mod = CatBoostRanker(**parameters)
    mod = CatBoost(parameters)
    
    mod.fit(train_pool, eval_set=test_pool, plot=True)
    
    return mod



In [325]:
# parameters = {'custom_metric': ['PrecisionAt:top=10', 'RecallAt:top=10', 'MAP:top=10']}
# parameters = {'custom_metric': ['PFound',  'AverageGain:top=10'],}
# parameters = {'custom_metric': ['NDCG',  'AverageGain:top=100'],}
# parameters={'learning_rate': .1,'custom_metric': ['NDCG:type=ranking',  'AverageGain:top=1000'],'eval_metric':'AUC:type=Ranking','one_hot_max_size': X_train['industy'].nunique()}
parameters={'learning_rate': .1,'custom_metric': ['NDCG', 'AverageGain:top=1000'],'eval_metric':'Logloss','one_hot_max_size': X_train['industy'].nunique()}

# 'YetiRank' 'RMSE'
model = fit_model('RMSE', train_p, test_p, parameters, verbose=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/target/target_converter.cpp:379: Target with classes must contain only 2 unique values for binary classification

In [302]:
# for i in range(10):
#     print(model.predict(Pool(X_test.iloc[i], cat_features=categorical_features_names)),y_test.iloc[i].values[0])

    
    

y_pred = model.predict(Pool(data=X_test.iloc[0:10], cat_features=categorical_features_names)  )  
rank = y_pred.argsort()
y_pred

array([-0.03088938,  0.11179415, -0.11789988,  0.05019727, -0.03518652,
        0.16457119,  0.01458887,  0.0849922 ,  0.03207335, -0.0489555 ])

In [303]:
rank

array([2, 9, 4, 0, 6, 8, 3, 7, 1, 5], dtype=int64)

In [278]:
y_test.iloc[0:10].values.squeeze().argsort()

array([0, 5, 1, 2, 3, 7, 4, 8, 9, 6], dtype=int64)

In [316]:
y_pred = model.predict(Pool(data=X_test.iloc[0:10], cat_features=categorical_features_names)  )  
rank = y_pred.argsort()
y_pred

array([-0.00058625,  0.00312672,  0.00098885,  0.0001612 , -0.00044387,
        0.00625504, -0.00032452,  0.00181612,  0.00017939,  0.00039243])

In [317]:
rank

array([0, 4, 6, 3, 8, 9, 2, 7, 1, 5], dtype=int64)

In [318]:
# model.get_feature_importance(train_p)
# feature_importances = model.get_feature_importance(train_p)
# for score, name in sorted(zip(feature_importances, X_train.columns), reverse=True):
#     print('{}: {}'.format(name, score))
######
# widget = MetricVisualizer(['model_depth_5', 'model_depth_8']) #事后对比，model_depth_5是‘train_dir’
# widget.start()
####
# model.save_model('catboost_model.dump')
# model = CatBoostClassifier()
# model.load_model('catboost_model.dump');
y_pred.argsort().argsort()

array([0, 8, 6, 3, 1, 9, 2, 7, 4, 5], dtype=int64)

In [319]:
y_test.iloc[0:10].values.squeeze().argsort().argsort()

array([0, 2, 3, 4, 6, 1, 9, 5, 7, 8], dtype=int64)